In [1]:
# libraries required
!pip install numpy
!pip install pandas
!pip install tensorflow==2.18.0

In [3]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)
# 파이썬 warninig 모듈은 경고메세지를 관리하는 기능을 제공
# ignore - 특정 종류의 경고를 무시, futruewarning - 미래버전에서 변경될 가능성이 있는 기능에 대한 경고
# 주로 pandas, tensorflow 에서 많이 사용함.

# 왜?
# 불필요한 경고 메세지를 숨기고, 로그가 지저분해지는것을 막기위해, 그치만 무시하는것이 마냥 좋진 않음


In [4]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
# let's check how many options we have
concrete_data.shape

(1030, 9)

In [6]:
# let's check the database for missing values
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [9]:
concrete_data.isnull().sum()
# pandas 에서 누락된 데이터를 계산하는 함수 -> 결측치 처리 함수

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


In [10]:
# split data into prediction and target
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
target = concrete_data['Strength']


In [11]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [12]:
target.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


In [13]:
# normalize - 데이터 정규화 과정
predictors_norm=(predictors-predictors.mean())/predictors.std()
# 각 열에서 해당열의 평균값을 빼준다. 이를 통해 데이터의 분포를 평균 0으로 만들고
# 평균이 0 이 된 데이터를 각열의 표준편차로 나누어줌
predictors_norm.head()
# 상위 5개의 행 표시

# 즉, 데이터 정규화란 -> 데이터 전처리 과정 중 하나로 각열의 데이터를 '표준화' 시켜 학습시 더 안정적이고 빠른 수렴을 도모하기 위한 전처리 과정이다.

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [15]:
n_cols = predictors_norm.shape[1]


In [16]:
n_cols

8

In [17]:
predictors_norm.shape

(1030, 8)

In [19]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense

In [20]:
def regression_model():
  model = Sequential()
  model.add(Input(shape=(n_cols,)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1))



# compile model
  model.compile(optimizer ='adam', loss='mean_squared_error')
  return model


In [21]:
# build model
model = regression_model()


In [22]:
# fit the model
model.fit(predictors_norm,target,validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 3s - 135ms/step - loss: 1637.9750 - val_loss: 1131.9384
Epoch 2/100
23/23 - 0s - 21ms/step - loss: 1501.6443 - val_loss: 996.0472
Epoch 3/100
23/23 - 0s - 5ms/step - loss: 1281.9138 - val_loss: 792.1542
Epoch 4/100
23/23 - 0s - 7ms/step - loss: 958.5759 - val_loss: 540.9168
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 590.1296 - val_loss: 317.7117
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 331.7758 - val_loss: 200.4553
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 241.5702 - val_loss: 171.7903
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 222.6691 - val_loss: 161.7864
Epoch 9/100
23/23 - 0s - 5ms/step - loss: 210.3486 - val_loss: 158.4651
Epoch 10/100
23/23 - 0s - 6ms/step - loss: 201.2812 - val_loss: 160.2304
Epoch 11/100
23/23 - 0s - 7ms/step - loss: 193.7167 - val_loss: 155.8360
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 187.3178 - val_loss: 155.1695
Epoch 13/100
23/23 - 0s - 6ms/step - loss: 181.8028 - val_loss: 154.6693
Epoch 14/100
23/23 - 0s - 6ms/step - loss: 175.8950 -

In [24]:
score = model.evaluate(predictors_norm, target, verbose=0)

In [28]:
print("accuracy :{}".format(score))


accuracy :69.90519714355469


In [29]:
from mmap import ACCESS_WRITE
# 5개의 은닉창, 각 50개의 노트와 relu 활성화 함수를 사용, 한개의 출력층, optimized 는 adam 사용
def new_regression():
  model = Sequential()
  input_new_value = predictors_norm.shape[1]
  model.add(Input(shape=(input_new_value,)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1),)

  # complie model
  model.compile(optimizer ='adam', loss='mean_squared_error')
  return model

In [30]:
# buil model
new_model = new_regression()

In [35]:
# fit model
new_model.fit(predictors_norm, target, validation_split=0.1 ,epochs=100, verbose=2)

socre = new_model.evaluate(predictors_norm,target, verbose=0.0)

Epoch 1/100
29/29 - 3s - 105ms/step - loss: 1501.5994 - val_loss: 974.3340
Epoch 2/100
29/29 - 0s - 6ms/step - loss: 761.1617 - val_loss: 150.1692
Epoch 3/100
29/29 - 0s - 6ms/step - loss: 270.9908 - val_loss: 169.4454
Epoch 4/100
29/29 - 0s - 11ms/step - loss: 204.7837 - val_loss: 161.7632
Epoch 5/100
29/29 - 0s - 8ms/step - loss: 185.2979 - val_loss: 161.2720
Epoch 6/100
29/29 - 0s - 4ms/step - loss: 172.7514 - val_loss: 145.2672
Epoch 7/100
29/29 - 0s - 9ms/step - loss: 159.3292 - val_loss: 147.7168
Epoch 8/100
29/29 - 1s - 18ms/step - loss: 146.2985 - val_loss: 135.6748
Epoch 9/100
29/29 - 0s - 4ms/step - loss: 136.0894 - val_loss: 128.5792
Epoch 10/100
29/29 - 0s - 4ms/step - loss: 123.7196 - val_loss: 119.3323
Epoch 11/100
29/29 - 0s - 4ms/step - loss: 113.6505 - val_loss: 120.6099
Epoch 12/100
29/29 - 0s - 4ms/step - loss: 102.5262 - val_loss: 106.4623
Epoch 13/100
29/29 - 0s - 5ms/step - loss: 90.9432 - val_loss: 101.4520
Epoch 14/100
29/29 - 0s - 4ms/step - loss: 79.4138 - val

In [36]:
print("accuracy{}".format(score))

accuracy69.90519714355469
